In [2]:
import torch

# 课程准备

## PyTorch基本使用

### 什么是张量

张量(tensor)是一个由数值构成的数组，可以有多个维度。

构建一个一维数组，其包含12个`int64`类型的元素

In [3]:
x: torch.Tensor = torch.arange(12)
print(f"shape: {x.shape}, number element: {x.numel()}, type of element: {x.dtype}")
print(x)

shape: torch.Size([12]), number element: 12, type of element: torch.int64
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])


可以使用`reshape`改变一个张量的形状

In [4]:
x = x.reshape(3, 4)
print("shape: ", x.shape)
print(x)

shape:  torch.Size([3, 4])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


### 常用的张量操作

#### `cat`
可以使用`cat`将多个有着相同形状(shape)的张量拼接起来，其中`dim`表示拼接的维度

比如现在有两个维度为$n$的张量$X$和$Y$，`dim`可取的范围为$[-n, n-1]$

$$
cat((X, Y), dim=d)=\begin{cases}
cat((X, Y), dim=d+n) &, d \lt 0 \\\\
X \ \mathbf{join} \ Y &, d=0 \\\\
X[i_0][i_1]\cdots[i_{d-1}]+Y[i_0][i_1]\cdots[i_{d-1}], i_k \in \{0, 1, 2, \cdots X.shape[k]-1\} &, d > 0
\end{cases}$$

其中

$$Z=X\ \mathbf{join} \ Y=[X[0], X[1], \cdots, Y[0], Y[1], \cdots]$$

In [16]:
X = torch.arange(12, dtype=torch.float32).reshape((2, 2, 3))
Y = torch.arange(12, 24, dtype=torch.float32).reshape((2, 2, 3))
print(f"X={X}\n Y={Y}")
torch.cat((X, Y), dim=2)

X=tensor([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.],
         [ 9., 10., 11.]]])
 Y=tensor([[[12., 13., 14.],
         [15., 16., 17.]],

        [[18., 19., 20.],
         [21., 22., 23.]]])


tensor([[[ 0.,  1.,  2., 12., 13., 14.],
         [ 3.,  4.,  5., 15., 16., 17.]],

        [[ 6.,  7.,  8., 18., 19., 20.],
         [ 9., 10., 11., 21., 22., 23.]]])

### 广播(broadcasting)

在正常情况，对两个张量执行任意操作的前提是两个张量有一样的形状。但是PyTorch允许两个张量即使形状不同也可以执行，这种机制称为[广播](https://numpy.org/doc/stable/user/basics.broadcasting.html#basics-broadcasting)。

```python
>>> a = np.array([1.0, 2.0, 3.0])
>>> b = np.array([2.0, 2.0, 2.0])
>>> a * b
array([2.,  4.,  6.])
```

`PyTorch`在执行张量的操作之前会检查两个张量的形状，从右往左一个一个元素比较

$$\begin{aligned}
A: a_0 \times &a_1 \cdots \times a_n \\
B: \ \ \ \ \ \ \ \ &b_0 \cdots \times b_n
\end{aligned}$$

只有下面两种情况，检查才会成功，否则会抛出`ValueError`的异常
- 元素相等
- 其中一个是1

当检查通过后，但是张量形状不一致时，会将张量广播成形状一致的张量，比如

$$\begin{aligned}
A&: 8 \times 1 \times 6 \times 1 \\
B&: \ \ \ \ \ \ \ 7 \times 1 \times 5 \\
Result&: 8\times7\times6\times5
\end{aligned}$$

对于需要广播的张量，比如维度$d_{old}$需要广播到$d_{new}$，那么会将$\{d_{old+1}, d_{old+2}, \cdots, d_{old+n-1}\}$复制$d_{new}-1$个

比如
```python
>>> a = np.array([[ 0.0,  0.0,  0.0],
...               [10.0, 10.0, 10.0],
...               [20.0, 20.0, 20.0],
...               [30.0, 30.0, 30.0]])
>>> b = np.array([1.0, 2.0, 3.0])
>>> a + b
array([[  1.,   2.,   3.],
        [11.,  12.,  13.],
        [21.,  22.,  23.],
        [31.,  32.,  33.]])
>>> b = np.array([1.0, 2.0, 3.0, 4.0])
>>> a + b
Traceback (most recent call last):
ValueError: operands could not be broadcast together with shapes (4,3) (4,)
```

![](../resources/broadcasting_1.png)

![](../resources/broadcasting_2.png)

In [10]:
a = torch.arange(3).reshape(3, 1)
b = torch.arange(2).reshape(1, 2)
print(f"a={a}\nb={b}\nbroadcasted_a={torch.cat((a, a), dim=1)}\nbroadcasted_b={torch.cat((b, b, b))}")
a + b

a=tensor([[0],
        [1],
        [2]])
b=tensor([[0, 1]])
broadcasted_a=tensor([[0, 0],
        [1, 1],
        [2, 2]])
broadcasted_b=tensor([[0, 1],
        [0, 1],
        [0, 1]])


tensor([[0, 1],
        [1, 2],
        [2, 3]])